In [161]:
import sqlite3
import pandas as pd

In [221]:
sql_query = """SELECT name FROM sqlite_master
    WHERE type='table';"""

In [313]:
professor_df = pd.read_excel("/Users/99ksh/TDB/professor.xlsx")
course_classed_df = pd.read_excel("/Users/99ksh/TDB/course_classed.xlsx")
norm_course_df = pd.read_excel("/Users/99ksh/TDB/norm_course.xlsx")
charge_df = pd.read_excel("/Users/99ksh/TDB/charge.xlsx")
university_df = pd.read_excel("/Users/99ksh/TDB/university.xlsx")

In [314]:
professor_df = professor_df.iloc[:, 1:]
course_classed_df = course_classed_df.iloc[:, 1:]
norm_course_df = norm_course_df.iloc[:, 1:]
charge_df = charge_df.iloc[:, 1:]
university_df = university_df.iloc[:, 1:]

In [315]:
university_df.rename(columns={"사립/국립 구분": "구분"}, inplace=True)
university_df.rename(columns={"대학소재지(시/도)": "대학소재지"}, inplace=True)

In [316]:
con = sqlite3.connect('kocw.db')

In [317]:
cur = con.cursor()
cur.execute(sql_query)
con.execute("PRAGMA foreign_keys = ON")
print(cur.fetchall())

[]


In [318]:
cur.execute("DROP TABLE IF EXISTS charge")
cur.execute("DROP TABLE IF EXISTS norm_course")
cur.execute("DROP TABLE IF EXISTS course_classed")
cur.execute("DROP TABLE IF EXISTS professor")
cur.execute("DROP TABLE IF EXISTS university")

In [319]:
university = """ CREATE TABLE university (
            제공대학명 CHAR(50) PRIMARY KEY NOT NULL UNIQUE,
            대학소재지 CHAR(50) ,
            구분 CHAR(50)
        ); """

In [320]:
professor = """ CREATE TABLE professor (
            교수코드 INTEGER PRIMARY KEY NOT NULL UNIQUE,
            담당교수명 CHAR(50) ,
            담당분야 CHAR(50),
            소속대학명 CHAR(50),
            FOREIGN KEY (소속대학명) REFERENCES university (제공대학명)
                ON UPDATE CASCADE
                ON DELETE SET NULL
        ); """

In [321]:
norm_course = """ CREATE TABLE norm_course (
    강의코드 INTEGER PRIMARY KEY NOT NULL UNIQUE,
    강의명 CHAR(50),
    교육분류 CHAR(50),
    강의년도 INTEGER,
    강의학기 INT,
    제공대학명 CHAR(50),
    제공일자 DATE,
    FOREIGN KEY (제공대학명) REFERENCES university (제공대학명)
    
        ); """

In [322]:
course_classed  = """ CREATE TABLE course_classed (
    강의명 CHAR(50) PRIMARY KEY NOT NULL UNIQUE,
    교육분류 CHAR(50)
        ); """

In [323]:
charge = """ CREATE TABLE charge (
    교수코드 INTEGER,
    강의코드 INTEGER,
    PRIMARY KEY (교수코드, 강의코드),
    FOREIGN KEY (교수코드) REFERENCES professor (교수코드)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (강의코드) REFERENCES norm_course (강의코드)
        ON UPDATE CASCADE
        ON DELETE CASCADE
        ); """

In [324]:
cur.execute(charge)
cur.execute(university)
cur.execute(professor)
cur.execute(norm_course)
cur.execute(course_classed)

In [325]:
cur.execute(sql_query)
cur.fetchall()

[('charge',),
 ('university',),
 ('professor',),
 ('norm_course',),
 ('course_classed',)]

In [326]:
professor_df.to_sql('professor', con, if_exists='replace', index=False)
course_classed_df.to_sql('course_classed', con, if_exists='replace', index=False)
norm_course_df.to_sql('norm_course', con, if_exists='replace', index=False)
charge_df.to_sql('charge', con, if_exists='replace', index=False)
university_df.to_sql('university', con, if_exists='replace', index=False)

DatabaseError: Execution failed on sql 'DROP TABLE "norm_course"': foreign key mismatch - "charge" referencing "professor"

In [175]:
conn.close

<function Connection.close()>

In [218]:
rows = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
tables = [row[0] for row in rows]
print(tables)

def sql_identifier(s):
    return '"' + s.replace('"', '""') + '"'

for table in tables:
    print("table: " + table)
    rows = cur.execute("PRAGMA table_info({})".format(sql_identifier(table)))
    print(rows.fetchall())
    rows = cur.execute("PRAGMA foreign_key_list({})".format(sql_identifier(table)))
    print(rows.fetchall())

['professor', 'course_classed', 'norm_course', 'charge', 'university']
table: professor
[(0, '교수코드', 'INTEGER', 0, None, 0), (1, '담당교수명', 'TEXT', 0, None, 0), (2, '담당분야', 'TEXT', 0, None, 0), (3, '제공대학명', 'TEXT', 0, None, 0)]
[]
table: course_classed
[(0, '강의명', 'TEXT', 0, None, 0), (1, '교육분류', 'TEXT', 0, None, 0)]
[]
table: norm_course
[(0, '강의코드', 'INTEGER', 0, None, 0), (1, '강의명', 'TEXT', 0, None, 0), (2, '강의년도', 'INTEGER', 0, None, 0), (3, '강의학기', 'INTEGER', 0, None, 0), (4, '제공대학명', 'TEXT', 0, None, 0), (5, '제공일자', 'TEXT', 0, None, 0)]
[]
table: charge
[(0, '교수코드', 'INTEGER', 0, None, 0), (1, '강의코드', 'INTEGER', 0, None, 0)]
[]
table: university
[(0, '제공대학명', 'TEXT', 0, None, 0), (1, '대학소재지(시/도)', 'TEXT', 0, None, 0), (2, '사립/국립 구분', 'TEXT', 0, None, 0)]
[]
